In [1]:
import pandas as pd
import json
import csv
import geopandas as gpd

In [2]:
path = '/Users/glennjames/Library/Mobile Documents/com~apple~CloudDocs/University/USyd/PLAN9071/Python/Data/MB_Data/Raw/'
output_path = '/Users/glennjames/Library/Mobile Documents/com~apple~CloudDocs/University/USyd/PLAN9071/Python/Data/MB_Data/'

In [3]:
# mb21_gdf = gpd.read_file(path + "MB_2021_AUST_GDA2020.json")

In [4]:
gdf = gpd.read_file(path + "MB_2016_NSW.json")

##### Plan:
- Filter 2021 mesh blocks to NSW
- Convert 2021 mesh blocks into points
- Join these to the 2016 mesh blocks according to where they are spatially within them
- Each 2016 mesh block should then have a dictionary of the corresponding 2021 mesh blocks

In [36]:
mb16_gdf = gdf
# mb16_gdf['geometry'] = mb16_gdf['geometry'].centroid

In [ ]:
# fitler nsw
mb21_nsw_gdf = mb21_gdf[mb21_gdf["STE_NAME21"] == 'New South Wales']

In [23]:
# convert to points
mb21_nsw_points = mb21_nsw_gdf
mb21_nsw_points['geometry'] = mb21_nsw_gdf['geometry'].centroid

/var/folders/p1/7llvcfs17mzb0d7gndybljth0000gn/T/ipykernel_70918/2352326737.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mb21_nsw_points['geometry'] = mb21_nsw_gdf['geometry'].centroid
/Users/glennjames/Library/Mobile Documents/com~apple~CloudDocs/University/USyd/PLAN9071/Python/.venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
## checking the coordinate systems, both are the same:
mb16_gdf.crs == mb21_nsw_gdf.crs


True

In [39]:
# use the within function
mb16_21 = gpd.sjoin(mb21_nsw_points, mb16_gdf, predicate='within')

In [37]:
print(mb16_gdf.geometry.type.unique())

[None 'Polygon']


In [ ]:
mb21_nsw_points.within(reference_polygon)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [46]:
mb16_21.geometry.type.unique()

array(['Point'], dtype=object)

In [47]:
mb16_21_poly = gpd.sjoin(mb16_gdf, mb21_nsw_points, predicate='contains')

In [54]:
# mb16_21_dict = {}
# for row, idx in mb16_21_poly[['MB_CODE16', 'MB_CODE21']].iterrows():
#     mb16_21_dict[row['MB_CODE16']] = row['MB_CODE21']



In [57]:
mb16_21_poly.to_file(output_path + "MB16_21_NSW.geojson", driver="GeoJSON")

In [60]:
mb16_21_mapping = dict(zip(mb16_21_poly['MB_CODE16'], mb16_21_poly['MB_CODE21']))